In [1]:
#抽主文
import os, re
import json
import jieba, numpy
import string, gensim
import threading
import pandas as pd
import numpy as np
from gensim.models.doc2vec import Doc2Vec
from collections import Counter

# 先只抓刑事判決
dirpath = os.environ['USERPROFILE'] + '\\Desktop\\酒駕\\'
x_train = []
cnt = 0

with open(os.environ['USERPROFILE'] + '\\Desktop\\stopword.txt', 'r', encoding = 'utf-8') as f:
    stopwords = f.read().split('\n')
g = open(os.environ['USERPROFILE'] + '\\Desktop\\result.txt', 'r', encoding = 'utf-8')

cas = 0    
    
for i in os.listdir(dirpath):
    with open(os.path.join(dirpath, i), "r", encoding = 'UTF-8') as f:
        data = json.load(f)['JFULL']
        data = data.replace(' ', '').replace('　', '')
    target0 = data.find('主文\r\n')    
    # 如果案字帶有秩的，代表為社會秩序維護法的案子，並不在我們的管理範圍內
    if "秩" in i: continue

    # 如果 JFULL 找不到主文，那就代表是 pdf 檔案
    # 暫定不處置
    if target0 == -1: continue     
    else:
        target1 = [9999999]
        if data.find('事實及理由\r\n') != -1:
            target1.append(data.find('事實及理由\r\n', target0))
        if data.find('犯罪事實\r\n') != -1:
            target1.append(data.find('犯罪事實\r\n', target0))
        if data.find('理由\r\n') != -1:
            target1.append(data.find('理由\r\n', target0))
        if data.find('事實\r\n') != -1:
            target1.append(data.find('事實\r\n', target0))
        if data.find('中華民國') != -1:
            target1.append(data.find('中華民國', target0))
        target1 = min(target1)
        if target1 == 9999999: continue
            
        data = data[target1:].replace('\r\n', '')
        data = re.split(r'[，。「」（）『』【】；：、]', data)
        jieba.load_userdict(os.environ['USERPROFILE'] + "\\Desktop\\private_data\\Sync\\TKU_LIFE_lab\\Dict.txt")
        for i in range(len(data)): 
#             data[i] = data[i].replace("，", " ").replace("。", " ").replace("「", " ").replace("」", " ").replace("（", " ") \
#                        .replace("）", " ").replace("【", " ").replace("】", " ").replace("、", " ").replace("：", " ") \
#                        .replace("；", " ")
            tmp = jieba.lcut(data[i])
            tmp = [w for w in tmp if w not in stopwords]
            TaggedDoc = gensim.models.doc2vec.TaggedDocument
            document = TaggedDoc(tmp, tags=[cnt])
            x_train.append(document)
            cnt += 1
        cas += 1
        if cas % 500 == 0: print(cas)
print('finished!')
        

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\nsl\\Desktop\\stopword.txt'

In [ ]:
print(x_train[1])

In [ ]:
def train(x_train):
    model = Doc2Vec(x_train, min_count = 1, window = 3, vector_size = 300, negative = 5, sample = 1e-3)
    model.train(x_train, total_examples = model.corpus_count, epochs = 10)
    return model

model = train(x_train)
model.save("model.bin")

In [ ]:
vector = model.infer_vector(doc)

for i in os.listdir(dirpath):
    with open(os.path.join(dirpath, i), "r", encoding = 'UTF-8') as f:
        data = json.load(f)['JFULL']
        data = data.replace(' ', '').replace('　', '')
    target0 = data.find('主文\r\n')    
    # 如果案字帶有秩的，代表為社會秩序維護法的案子，並不在我們的管理範圍內
    if "秩" in i: continue

    # 如果 JFULL 找不到主文，那就代表是 pdf 檔案
    # 暫定不處置
    if target0 == -1: continue     
    else:
        target1 = []
        if data.find('事實及理由\r\n') != -1:
            target1.append(data.find('事實及理由\r\n', target0))
        if data.find('犯罪事實\r\n') != -1:
            target1.append(data.find('犯罪事實\r\n', target0))
        if data.find('理由\r\n') != -1:
            target1.append(data.find('理由\r\n', target0))
        if data.find('事實\r\n') != -1:
            target1.append(data.find('事實\r\n', target0))
        if data.find('中華民國') != -1:
            target1.append(data.find('中華民國', target0))
        target1 = min(target1)

        data = data.translate(str.maketrans('', '', string.punctuation))
        data = data.replace("，", " ").replace("。", " ").replace("「", " ").replace("」", " ").replace("（", " ") \
                   .replace("）", " ").replace("【", " ").replace("】", " ").replace("、", " ").replace("：", " ") \
                   .replace("；", " ").replace("\r\n", "")
        
